Custom Validators

Field-Level Validators

In [ ]:
from pydantic import BaseModel, Field, ValidationError

class User(BaseModel):
    name: str
    age: int = Field(gt=21)
    
try:
    user = User(name="Alice", age=22)
    print(user)
except ValidationError as e:
    print(e)

name='Alice' age=22


In [4]:
from pydantic import field_validator

class User(BaseModel):
    name: str
    age: int = Field(gt=0)

    @field_validator("age")
    @classmethod
    def validate_age(cls, v):
        if v < 18 or v % 2 != 0:
            raise ValueError("Age must be even and at least 18.")
        return v

try:
    user = User(name="Alice", age=22)
    # would raise an exception because age is odd
    # user = User(name="Alice", age=21)
    print(user)
except ValueError as e:
    print(e)

name='Alice' age=22


Model-Level Validators

In [6]:
from datetime import date
from pydantic import BaseModel, ValidationError, model_validator

class ScheduledCourse(BaseModel):
    department: str
    course_number: int
    start_date: date
    end_date: date
    
    @model_validator(mode="before")
    @classmethod
    def validate_dates(cls, data: dict):
        if data["start_date"] > data["end_date"]:
            raise ValueError("Start date must be before end date for the course.")
        return data

try:
    # custom validator is executed before instantiating an object
    # because it is a "before" validation. So custom validation exception
    # is thrown even if not all mandatory fields are filled in
    course = ScheduledCourse(
        start_date="2025-12-22",
        end_date="2025-11-10"
    )
    print(course)
except ValidationError as e:
    print(e)

1 validation error for ScheduledCourse
  Value error, Start date must be before end date for the course. [type=value_error, input_value={'start_date': '2025-12-2...end_date': '2025-11-10'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [11]:
from datetime import date
from pydantic import BaseModel, ValidationError, model_validator

class ScheduledCourse(BaseModel):
    department: str
    course_number: int
    start_date: date
    end_date: date
    
    @model_validator(mode="after")
    def validate_dates(self):
        if self.start_date > self.end_date:
            raise ValueError("Start date must be before end date for the course.")

try:
    # we need to provide all mandatory values here in order to run custom validation
    # because it is in "after" mode, i.e. it needs an instance to be run against
    course = ScheduledCourse(
        department="cs",
        course_number=101,
        start_date="2025-12-22",
        end_date="2025-11-10"
    )
    print(course)
except ValidationError as e:
    print(e)

1 validation error for ScheduledCourse
  Value error, Start date must be before end date for the course. [type=value_error, input_value={'department': 'cs', 'cou...end_date': '2025-11-10'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error
